In [2]:
import os
import json
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from sklearn.model_selection import train_test_split

In [ ]:
# 1. 데이터셋 클래스 정의
class ClothingDataset(Dataset):
    def __init__(self, root_dir, label_dir, transform=None):
        self.root_dir = root_dir
        self.label_dir = label_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        self._load_dataset()

    def _load_dataset(self):
        categories = ["아우터", "하의", "원피스", "상의"]

        # 라벨링 데이터 폴더 내의 모든 파일에 대해 처리
        for style_folder in os.listdir(self.label_dir):
            style_path = os.path.join(self.label_dir, style_folder)
            if not os.path.isdir(style_path):
                continue

            for label_file in os.listdir(style_path):
                if label_file.endswith('.json'):
                    label_path = os.path.join(style_path, label_file)

                    with open(label_path, 'r', encoding='utf-8') as f:
                        label_data = json.load(f)

                    # 이미지 식별자를 기반으로 이미지 파일 경로 생성
                    image_id = label_data["이미지 정보"]["이미지 식별자"]
                    image_file = f"{image_id}.jpg"
                    image_path = os.path.join(self.root_dir, style_folder, image_file)

                    # 라벨 추출
                    label = None
                    for category in categories:
                        if label_data["데이터셋 정보"]["데이터셋 상세설명"]["라벨링"][category] != [{}]:
                            label = category
                            break

                    if label is not None and os.path.exists(image_path):
                        self.image_paths.append(image_path)
                        self.labels.append(label)
                    else:
                        print(f"이미지 경로 또는 라벨이 유효하지 않음: {image_path}, 라벨: {label}")  # 문제 발생 시 출력

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        label_map = {"아우터": 0, "하의": 1, "원피스": 2, "상의": 3}
        return image, label_map[label]

In [ ]:
# 2. 데이터 증강 포함한 변환 설정
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),  # 50% 확률로 좌우 반전
    transforms.RandomRotation(degrees=5),   # -15도에서 15도 사이에서 회전
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.2, hue=0.05),  # 색상 변화
    transforms.RandomResizedCrop(size=224, scale=(0.9, 1.1)),  # 이미지 확대 및 축소
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
current_dir = os.getcwd()
# 3. 데이터셋 생성
root_dir = os.path.join(current_dir, 'New_sample/원천데이터/원천데이터_1')
label_dir = os.path.join(current_dir, 'New_sample/라벨링데이터')
full_dataset = ClothingDataset(root_dir=root_dir, label_dir=label_dir, transform=transform)

C:\SS\AI_pj
이미지 경로 또는 라벨이 유효하지 않음: C:\SS\AI_pj\New_sample/원천데이터/원천데이터_1\로맨틱\165.jpg, 라벨: 아우터


In [ ]:
# 4. 학습 및 검증 데이터셋으로 분리
train_indices, val_indices = train_test_split(list(range(len(full_dataset))), test_size=0.2, random_state=42)
train_dataset = Subset(full_dataset, train_indices)
val_dataset = Subset(full_dataset, val_indices)

# 5. DataLoader 생성
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# 6. 모델 정의

import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(in_features=32 * 56 * 56, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.fc3 = nn.Linear(in_features=60, out_features=num_classes)
        self.dropout = nn.Dropout(0.5)  # 드롭아웃 추가

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # 드롭아웃 적용
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [1]:
# 7. 모델 초기화
model = SimpleCNN(num_classes=4)

# 8. GPU가 사용 가능하면 모델을 GPU로 이동
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 9. 손실 함수 및 옵티마이저 정의
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

early_stopping_patience = 3  # 성능이 개선되지 않는 에포크 수
min_delta = 0.01  # 개선으로 간주될 최소 변화량
best_val_accuracy = 0.0
patience_counter = 0

NameError: name 'SimpleCNN' is not defined

In [ ]:
# 10. 학습 및 검증
from tqdm import tqdm

num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataloader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}: Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # 검증 과정
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for data in val_dataloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss /= len(val_dataloader)
    val_accuracy = 100 * val_correct / val_total
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Early Stopping 체크
    if val_accuracy - best_val_accuracy > min_delta:
        best_val_accuracy = val_accuracy
        patience_counter = 0  # 개선이 이루어졌으므로 카운터 초기화
    else:
        patience_counter += 1  # 개선되지 않았을 경우 카운터 증가

    if patience_counter >= early_stopping_patience:
        print("Early stopping triggered")
        break  # 학습 중단

print('Finished Training')

Epoch 1/50: 100%|██████████████████████████████████████████████████████████████████████| 55/55 [09:04<00:00,  9.90s/it]


Epoch 1: Loss: 1.2363, Accuracy: 41.90%
Validation Loss: 1.0864, Validation Accuracy: 37.95%


Epoch 2/50:   2%|█▎                                                                     | 1/55 [00:10<09:49, 10.91s/it]

In [ ]:
# 예측한 모델을 활용하는 방법
from PIL import Image
import torch

def predict_image(image_path, model, transform, label_map):
    """
    주어진 이미지에 대해 모델이 예측한 라벨을 반환하는 함수.

    Parameters:
    - image_path: 예측할 이미지 파일 경로
    - model: 학습된 PyTorch 모델
    - transform: 이미지 전처리(transform)
    - label_map: 정수 라벨을 실제 클래스 이름으로 매핑하는 딕셔너리

    Returns:
    - predicted_label: 예측된 클래스 이름
    """
    # 모델을 평가 모드로 설정
    model.eval()

    # 이미지를 불러와서 전처리
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # 배치 차원을 추가

    # 모델을 사용하여 예측 수행
    with torch.no_grad():
        image = image.to(device)
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    # 예측된 라벨의 인덱스를 실제 클래스 이름으로 변환
    predicted_label = label_map[predicted.item()]
    return predicted_label


In [ ]:
# 라벨 맵핑 정의 (숫자 라벨을 클래스 이름으로 변환)
label_map = {0: "아우터", 1: "하의", 2: "원피스", 3: "상의"}


# 예측 수행
predicted_label1 = predict_image('C:/SS/AI_pj/New_sample/확인용데이터/상의.jpg', model, transform, label_map)
predicted_label2 = predict_image('C:/SS/AI_pj/New_sample/확인용데이터/하의.jpg', model, transform, label_map)
predicted_label3 = predict_image('C:/SS/AI_pj/New_sample/확인용데이터/원피스.jpg', model, transform, label_map)
predicted_label4 = predict_image('C:/SS/AI_pj/New_sample/확인용데이터/아우터.jpg', model, transform, label_map)


# 예측 결과 출력
print(f"상의의 Predicted Label: {predicted_label1}")
print(f"하의의 Predicted Label: {predicted_label2}")
print(f"원피스의 Predicted Label: {predicted_label3}")
print(f"아우터의 Predicted Label: {predicted_label4}")


상의의 Predicted Label: 상의
하의의 Predicted Label: 원피스
원피스의 Predicted Label: 원피스
아우터의 Predicted Label: 하의
